In [ ]:
#importiing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#files were directly uploaded to google colab. The following code can also be used to import the file
#from google.colab import files
#uploaded = files.upload()
df = pd.read_csv('weather_train.csv')

In [ ]:
#converting date and time from string to datetime to analyse those
from datetime import datetime
df['DATE (YYYY/MM/DD)'] = pd.to_datetime(df['DATE (YYYY/MM/DD)'])
df['DATE (YYYY/MM/DD)'] = df['DATE (YYYY/MM/DD)'].dt.month*30 + df['DATE (YYYY/MM/DD)'].dt.day
df['MST'] = pd.to_datetime(df['MST'])
df['MST'] = df['MST'].dt.hour*60 + df['MST'].dt.minute

In [ ]:
#converting pandas database to numpy array to employ keras
dfv = df.values
X = dfv[:, 0:18]
y = dfv[:, 18]
#converting temp into kelvin as i felt that negative temperatures might be problematic in establishing corellations
df['Tower Dry Bulb Temp [deg C]'] = df['Tower Dry Bulb Temp [deg C]'] + 273

In [ ]:
#preprocessing and scaling before training the nueral network
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 40)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.3)

In [ ]:
#importing model from keras 
from keras.models import Sequential
from keras.layers import Dense
model = Sequential([
    Dense(32, activation='relu', input_shape=(18,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

from keras import backend as K

#creating a function for R2 values for keras
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[coeff_determination])

In [ ]:
#training the model
hist = model.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_val, Y_val))

Epoch 1/100
329/329 [==============================] - 1s 3ms/step - loss: 60962.5586 - coeff_determination: -0.0724 - val_loss: 60964.1992 - val_coeff_determination: -0.0615
Epoch 2/100
329/329 [==============================] - 1s 2ms/step - loss: 60958.9648 - coeff_determination: -0.0794 - val_loss: 60956.0195 - val_coeff_determination: -0.1714
Epoch 3/100
329/329 [==============================] - 1s 2ms/step - loss: 60953.9766 - coeff_determination: -0.1445 - val_loss: 60955.2266 - val_coeff_determination: -0.1555
Epoch 4/100
329/329 [==============================] - 1s 2ms/step - loss: 60952.8086 - coeff_determination: -0.1415 - val_loss: 60955.7578 - val_coeff_determination: -0.1315
Epoch 5/100
329/329 [==============================] - 1s 2ms/step - loss: 60952.2344 - coeff_determination: -0.1089 - val_loss: 60957.6133 - val_coeff_determination: -0.0833
Epoch 6/100
329/329 [==============================] - 1s 2ms/step - loss: 60951.8867 - coeff_determination: -0.0928 - val_lo

In [ ]:
#evalueating the model
model.evaluate(X_test, Y_test)[1]

43/43 [==============================] - 0s 2ms/step - loss: 47420.9648 - coeff_determination: -0.0079


-0.007879135198891163

In [ ]:
test = pd.read_csv('weather_test.csv')

In [ ]:
test['DATE (YYYY/MM/DD)'] = pd.to_datetime(test['DATE (YYYY/MM/DD)'])
test['DATE (YYYY/MM/DD)'] = test['DATE (YYYY/MM/DD)'].dt.month*30 + test['DATE (YYYY/MM/DD)'].dt.day
test['MST'] = pd.to_datetime(test['MST'])
test['MST'] = test['MST'].dt.hour*60 + test['MST'].dt.minute
x = test.iloc[:, 0:18].values
x_scaled = min_max_scaler.fit_transform(x)
#predicting the values of snowfall of the test set
prediction = model.predict(x_scaled)

In [ ]:
#converting the dataframe into a csv file
predictions = pd.DataFrame(prediction, columns=['Snow Depth [cm]']).to_csv('predictions.csv')